<a href="https://colab.research.google.com/github/AbdulSheffa/IRP/blob/main/FYP_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix


fatal: destination path 'pytorch-CycleGAN-and-pix2pix' already exists and is not an empty directory.
/content/pytorch-CycleGAN-and-pix2pix


In [2]:
!pip install -r requirements.txt


In [8]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
from PIL import Image
from models.networks import ResnetGenerator, NLayerDiscriminator

# ------------------------------
# 1. Dataset with Self-Supervised Learning & Masking
# ------------------------------
class RainDatasetSSL(Dataset):
    def __init__(self, dataset_name, input_dir, target_dir, transform=None, sample_size=None, output_dir=None, save_masked=False, epoch=0):
        self.dataset_name = dataset_name
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.output_dir = output_dir
        self.save_masked = save_masked
        self.epoch = epoch

        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))

        if sample_size:
            indices = random.sample(range(len(self.input_images)), sample_size)
            self.input_images = [self.input_images[i] for i in indices]
            self.target_images = [self.target_images[i] for i in indices]

        if save_masked and output_dir:
            os.makedirs(output_dir, exist_ok=True)
            self.masked_images_to_save = random.sample(self.input_images, min(10, len(self.input_images)))

    def __len__(self):
        return len(self.input_images)

    def mask_image(self, img_tensor, img_name):
        """Apply dynamic masking for self-supervised learning & save masked image if required."""
        if img_tensor.dim() == 4:  # If batch, process each image separately
            return torch.stack([self.mask_image(single_img, img_name) for single_img in img_tensor])

        if img_tensor.dim() != 3:
            raise ValueError(f"Unexpected tensor shape {img_tensor.shape}, expected (C, H, W)")

        _, h, w = img_tensor.shape
        mask_size = (random.randint(h // 6, h // 4), random.randint(w // 6, w // 4))
        x, y = random.randint(0, h - mask_size[0]), random.randint(0, w - mask_size[1])

        masked_tensor = img_tensor.clone()
        masked_tensor[:, x:x + mask_size[0], y:y + mask_size[1]] = -1

        # ✅ Ensure the output directory exists before saving
        if self.save_masked and self.output_dir:
            os.makedirs(self.output_dir, exist_ok=True)  # ✅ Ensure the directory exists

            save_path = os.path.join(self.output_dir, f"masked_{self.dataset_name}_epoch_{self.epoch}_{img_name}")
            save_image((masked_tensor + 1) / 2, save_path)  # Denormalizing before saving
            print(f"✅ Saved masked image: {save_path}")

        return masked_tensor


    def __getitem__(self, idx):
        input_img_path = os.path.join(self.input_dir, self.input_images[idx])
        target_img_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_img_path).convert("RGB")
        target_image = Image.open(target_img_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        masked_image = self.mask_image(input_image.clone(), self.input_images[idx])
        return masked_image, input_image, target_image, self.input_images[idx]  # ✅ Ensure 4 elements

# ------------------------------
# 2. Masked GAN-Based DeRain CycleGAN (Without Self-Attention)
# ------------------------------
class DeRainCycleGANSSL:
    def __init__(self, input_nc, output_nc, ngf, ndf, device):
        self.device = device
        self.netG_A = ResnetGenerator(input_nc, output_nc, ngf).to(device)
        self.netG_B = ResnetGenerator(output_nc, input_nc, ngf).to(device)
        self.netD_A = NLayerDiscriminator(output_nc, ndf).to(device)
        self.netD_B = NLayerDiscriminator(input_nc, ndf).to(device)

        self.criterionGAN = nn.MSELoss()
        self.criterionCycle = nn.L1Loss()
        self.criterionPerceptual = nn.MSELoss()

        self.vgg19 = models.vgg19(pretrained=True).features[:23].eval().to(device)
        for param in self.vgg19.parameters():
            param.requires_grad = False

        self.optimizer_G = torch.optim.Adam(
            list(self.netG_A.parameters()) + list(self.netG_B.parameters()), lr=0.00002, betas=(0.5, 0.999)
        )
        self.optimizer_D = torch.optim.Adam(
            list(self.netD_A.parameters()) + list(self.netD_B.parameters()), lr=0.00002, betas=(0.5, 0.999)
        )

    def perceptual_loss(self, x, y):
        return self.criterionPerceptual(self.vgg19(x), self.vgg19(y))

    def forward(self, real_A, real_B):
        fake_B = self.netG_A(real_A)
        rec_A = self.netG_B(fake_B)
        fake_A = self.netG_B(real_B)
        rec_B = self.netG_A(fake_A)
        return fake_B, rec_A, fake_A, rec_B

    def optimize_parameters(self, real_A, real_B, scaler):
        """Optimizes generator and discriminator parameters for one iteration."""

        self.optimizer_G.zero_grad()
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            real_A, real_B = real_A.half(), real_B.half()
            fake_B, rec_A, fake_A, rec_B = self.forward(real_A, real_B)
            loss_G_A = self.criterionGAN(self.netD_A(fake_B), torch.ones_like(self.netD_A(fake_B)))
            loss_G_B = self.criterionGAN(self.netD_B(fake_A), torch.ones_like(self.netD_B(fake_A)))
            loss_cycle_A = self.criterionCycle(rec_A, real_A) * 10.0
            loss_cycle_B = self.criterionCycle(rec_B, real_B) * 10.0
            loss_perceptual = self.perceptual_loss(fake_B, real_B) * 0.05
            loss_G = loss_G_A + loss_G_B + loss_cycle_A + loss_cycle_B + 0.001 * loss_perceptual

        scaler.scale(loss_G).backward()
        scaler.step(self.optimizer_G)
        scaler.update()

        self.optimizer_D.zero_grad()
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            loss_D_A = self.criterionGAN(self.netD_A(real_B.half()), torch.ones_like(self.netD_A(real_B.half())))
            loss_D_B = self.criterionGAN(self.netD_B(real_A.half()), torch.ones_like(self.netD_B(real_A.half())))
            loss_D = (loss_D_A + loss_D_B) * 0.3

        scaler.scale(loss_D).backward()
        scaler.step(self.optimizer_D)
        scaler.update()

        return loss_G.detach(), loss_D_A.detach(), loss_D_B.detach()


In [12]:
import os
import torch
import torch.nn as nn
import torch.cuda.amp as amp  # ✅ Import amp explicitly
from torchvision import transforms

# ✅ Initialize Mixed Precision Training
scaler = amp.GradScaler()  # ✅ Fix: Now `amp` is properly defined

# Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Initialize Model BEFORE Loading Checkpoints
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=64, ndf=64, device=device)

# ✅ Initialize Optimizers and Schedulers
# scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=20, gamma=0.5)
# scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=20, gamma=0.5)
scheduler_G = torch.optim.lr_scheduler.ReduceLROnPlateau(model.optimizer_G, mode='min', patience=5, factor=0.5)
scheduler_D = torch.optim.lr_scheduler.ReduceLROnPlateau(model.optimizer_D, mode='min', patience=5, factor=0.5)



# ✅ Define Checkpoint Directory
checkpoint_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)  # Ensure checkpoint folder exists

# ✅ Define Checkpoint Loading Function (Fixing NameError)
def load_latest_checkpoint(model, optimizer_G, optimizer_D, scheduler_G, scheduler_D, scaler, checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pth")]
    if not checkpoints:
        print("⚠️ No checkpoint found, starting from scratch.")
        return 0  # Start from epoch 0

    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("_")[-1].split(".")[0]))[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)

    print(f"🔄 Resuming training from {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.netG_A.load_state_dict(checkpoint['model_G_A_state_dict'])
    model.netG_B.load_state_dict(checkpoint['model_G_B_state_dict'])
    model.netD_A.load_state_dict(checkpoint['model_D_A_state_dict'])
    model.netD_B.load_state_dict(checkpoint['model_D_B_state_dict'])
    optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
    optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    scheduler_G.load_state_dict(checkpoint['scheduler_G_state_dict'])
    scheduler_D.load_state_dict(checkpoint['scheduler_D_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])

    return checkpoint['epoch'] + 1  # Resume from the next epoch

# ✅ Now Load the Latest Checkpoint AFTER Model Initialization
start_epoch = load_latest_checkpoint(model, model.optimizer_G, model.optimizer_D, scheduler_G, scheduler_D, scaler, checkpoint_dir)

# ✅ Now Training Will Continue From the Latest Saved Epoch
print(f"🚀 Training will start from epoch {start_epoch}")


<ipython-input-12-2796430268e6>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()  # ✅ Fix: Now `amp` is properly defined


Using device: cuda
🔄 Resuming training from /content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoints/checkpoint_epoch_79.pth


<ipython-input-12-2796430268e6>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


🚀 Training will start from epoch 80


In [9]:
import os
import torch
import torch.nn as nn
import torch.cuda.amp as amp  # Mixed Precision Training
from torchvision.transforms.functional import to_pil_image  # For correct image saving
from torchvision import transforms
from torch.utils.data import DataLoader
from models.networks import ResnetGenerator, NLayerDiscriminator

# Define dataset paths and batch sizes
datasets = {
    # "Rain100L": {
    #     "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/input",
    #     "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/target",
    #     "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_8/",
    #     "batch_size": 8,  # Custom batch size per dataset
    #     "sample_size": 100  # Take only 100 samples
    # },
    "DID-MDN-Heavy": {
        "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/rainy",
        "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/non_rainy",
        "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/",
        "batch_size": 4,
        "sample_size": 200  # Take 200 samples
    }
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Medium-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # },
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Light-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # }
}

# Image transformations with fixed normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ensures uniform image size
    transforms.CenterCrop((224, 224)),  # Keeps input-output pairs aligned
    transforms.RandomHorizontalFlip(p=0.5),  # Augmentation
    transforms.ColorJitter(brightness=0.02, contrast=0.02, saturation=0.02),  # Subtle augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def denormalize(tensor):
    """Convert [-1,1] normalized tensor back to [0,1] for visualization."""
    mean = torch.tensor([0.5, 0.5, 0.5], device=tensor.device).view(3, 1, 1)
    std = torch.tensor([0.5, 0.5, 0.5], device=tensor.device).view(3, 1, 1)
    return torch.clamp(tensor * std + mean, 0, 1)  # ✅ Ensure valid pixel range

# Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=64, ndf=64, device=device)

# Learning Rate Schedulers
scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=20, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=20, gamma=0.5)

# Training Configuration
num_epochs = 70
checkpoint_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)  # Ensure checkpoint folder exists

# Mixed Precision Training
scaler = torch.amp.GradScaler("cuda")

# Loop Through Each Dataset and Train
for dataset_name, paths in datasets.items():
    print(f"\n🔹 Training on {dataset_name} dataset...")

    input_dir = paths["input"]
    target_dir = paths["target"]
    output_dir = paths["output"]
    batch_size = paths["batch_size"]
    sample_size = paths["sample_size"]

    os.makedirs(output_dir, exist_ok=True)

    # Load dataset with masked saving enabled
    train_loader = DataLoader(
        RainDatasetSSL(
            dataset_name=dataset_name,
            input_dir=input_dir,
            target_dir=target_dir,
            transform=transform,
            sample_size=sample_size,
            output_dir=output_dir,
            save_masked=True
        ),
        batch_size=batch_size,
        shuffle=True
    )

    for epoch in range(num_epochs):
        for i, (masked_A, real_A, real_B, img_name) in enumerate(train_loader):
            masked_A, real_A, real_B = masked_A.to(device), real_A.to(device), real_B.to(device)

            # Optimize model and check loss
            loss_G, loss_D_A, loss_D_B = model.optimize_parameters(masked_A, real_B, scaler)

            # ✅ Ensure all loss values are valid tensors
            if not isinstance(loss_G, torch.Tensor) or not isinstance(loss_D_A, torch.Tensor) or not isinstance(loss_D_B, torch.Tensor):
                print(f"⚠️ Error: Loss values must be tensors! Received: loss_G={loss_G}, loss_D_A={loss_D_A}, loss_D_B={loss_D_B}")
                continue

            # ✅ Check for NaN values in loss
            if torch.isnan(loss_G).any() or torch.isnan(loss_D_A).any() or torch.isnan(loss_D_B).any():
                print("⚠️ NaN detected in loss! Skipping update")
                continue

            print(f"Epoch {epoch}/{num_epochs}, Batch {i}/{len(train_loader)}, Image: {img_name[0]}, "
                  f"Loss_G: {loss_G.item():.4f}, Loss_D_A: {loss_D_A.item():.4f}, Loss_D_B: {loss_D_B.item():.4f}")

        scheduler_G.step(loss_G)
        scheduler_D.step((loss_D_A + loss_D_B) * 0.5)  # ✅ Fix NameError


        torch.cuda.empty_cache()  # Prevent OOM errors

        # ✅ Save checkpoint every 5 epochs
        if epoch % 5 == 0 or epoch == num_epochs - 1:
            torch.save({
                'epoch': epoch,
                'model_G_A_state_dict': model.netG_A.state_dict(),
                'model_G_B_state_dict': model.netG_B.state_dict(),
                'model_D_A_state_dict': model.netD_A.state_dict(),
                'model_D_B_state_dict': model.netD_B.state_dict(),
                'optimizer_G_state_dict': model.optimizer_G.state_dict(),
                'optimizer_D_state_dict': model.optimizer_D.state_dict(),
                'scheduler_G_state_dict': scheduler_G.state_dict(),
                'scheduler_D_state_dict': scheduler_D.state_dict(),
                'scaler_state_dict': scaler.state_dict()
            }, os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth"))
            print(f"✅ Checkpoint saved at epoch {epoch}")

    # ✅ Save final generated images with correct colors
    if epoch == num_epochs - 1:
        model.netG_A.eval()
        with torch.no_grad():
            for i, (_, real_A, _, img_names) in enumerate(train_loader):
                real_A = real_A.to(device)
                fake_B = model.netG_A(real_A)
                fake_B = denormalize(fake_B)

                for j in range(real_A.size(0)):
                    img_name = img_names[j]
                    save_path = os.path.join(output_dir, f"{dataset_name}_epoch_{epoch}_de_rained_{img_name}")

                    # ✅ Convert to PIL and save
                    pil_img = to_pil_image(denormalize(fake_B[j].cpu()).squeeze(0))  # ✅ Ensure (C, H, W)  # ✅ Fix: Remove batch dim
                    pil_img.save(save_path)

                    print(f"✅ Saved: {save_path}")

print("Training on all datasets completed ✅")


Streaming output truncated to the last 5000 lines.
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/masked_DID-MDN-Heavy_epoch_0_rainy_228.jpg
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/masked_DID-MDN-Heavy_epoch_0_rainy_2196.jpg
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/masked_DID-MDN-Heavy_epoch_0_rainy_3372.jpg
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/masked_DID-MDN-Heavy_epoch_0_rainy_111.jpg
Epoch 41/60, Batch 31/50, Image: rainy_228.jpg, Loss_G: 2.3948, Loss_D_A: 0.0024, Loss_D_B: 0.0027
✅ Saved masked image: /content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_1/masked_DID-MDN-Heavy_epoch_0_rainy_3

In [6]:
import torch
import gc

torch.cuda.empty_cache()  # ✅ Releases unreferenced GPU memory
gc.collect()  # ✅ Clears Python garbage collector


3877

In [11]:
import os
import torch
import torch.nn as nn
from torchvision.utils import save_image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# ------------------------------
# 🔹 Improved Autoencoder with Multi-Scale Features
# ------------------------------
class Autoencoder(nn.Module):
    def __init__(self, input_nc):
        super(Autoencoder, self).__init__()

        # 🔹 Encoder
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder4 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))

        # 🔹 Bottleneck
        self.bottleneck = nn.Sequential(nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))

        # 🔹 Decoder with Multi-Scale Fusion
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(512 + 512, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder5 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1), nn.Sigmoid())  # ✅ Changed to Sigmoid

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        b = self.bottleneck(e4)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e4], dim=1))  # Multi-Scale Skip Connection
        d3 = self.decoder3(torch.cat([d2, e3], dim=1))
        d4 = self.decoder4(torch.cat([d3, e2], dim=1))
        d5 = self.decoder5(torch.cat([d4, e1], dim=1))
        return d5

# ------------------------------
# 🔹 SSIM Loss for Structural Similarity
# ------------------------------
import torch.nn.functional as F

class SSIMLoss(nn.Module):
    def __init__(self):
        super(SSIMLoss, self).__init__()

    def forward(self, img1, img2):
        C1 = 0.01 ** 2
        C2 = 0.03 ** 2

        mu1 = F.avg_pool2d(img1, 3, 1)
        mu2 = F.avg_pool2d(img2, 3, 1)
        sigma1 = F.avg_pool2d(img1 ** 2, 3, 1) - mu1 ** 2
        sigma2 = F.avg_pool2d(img2 ** 2, 3, 1) - mu2 ** 2
        sigma12 = F.avg_pool2d(img1 * img2, 3, 1) - mu1 * mu2

        ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1 + sigma2 + C2))
        return torch.clamp((1 - ssim).mean(), 0, 1)

# ------------------------------
# 🔹 Update Loss Function
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(input_nc=3).to(device)

mse_loss = nn.MSELoss()
ssim_loss = SSIMLoss().to(device)
perceptual_loss = PerceptualLoss().to(device)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0002)

# ------------------------------
# 🔹 Updated Training Loop with SSIM Loss
# ------------------------------
epochs = 60
for epoch in range(epochs):
    for images, filenames in train_loader:
        images = images.to(device)
        reconstructed = autoencoder(images)

        mse_loss_value = mse_loss(reconstructed, images)
        ssim_loss_value = ssim_loss(reconstructed, images)
        perceptual_loss_value = perceptual_loss(reconstructed, images)

        total_loss = mse_loss_value + 0.01 * perceptual_loss_value + 0.1 * ssim_loss_value  # ✅ Adjusted weights

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{epochs}], MSE: {mse_loss_value.item():.4f}, SSIM: {ssim_loss_value.item():.4f}, Perceptual: {perceptual_loss_value.item():.4f}")

# ------------------------------
# 🔹 Save Refined Images
# ------------------------------
output_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/Refined_Autoencoder_Results/results_2"
os.makedirs(output_dir, exist_ok=True)

for filename, reconstructed_image in reconstructed_images.items():
    save_path = os.path.join(output_dir, f"{filename.split('.')[0]}_refined.png")
    save_image(reconstructed_image, save_path)

print("\n🎯 Autoencoder Refinement Completed!")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/60], MSE: 0.0388, SSIM: 0.4546, Perceptual: 2.0103
Epoch [2/60], MSE: 0.0259, SSIM: 0.4571, Perceptual: 1.8435
Epoch [3/60], MSE: 0.0082, SSIM: 0.3627, Perceptual: 1.5472
Epoch [4/60], MSE: 0.0056, SSIM: 0.2485, Perceptual: 1.0331
Epoch [5/60], MSE: 0.0042, SSIM: 0.2689, Perceptual: 1.1350
Epoch [6/60], MSE: 0.0023, SSIM: 0.1457, Perceptual: 0.5487
Epoch [7/60], MSE: 0.0025, SSIM: 0.1788, Perceptual: 0.7137
Epoch [8/60], MSE: 0.0042, SSIM: 0.2248, Perceptual: 0.9093
Epoch [9/60], MSE: 0.0027, SSIM: 0.1506, Perceptual: 0.5891
Epoch [10/60], MSE: 0.0029, SSIM: 0.1751, Perceptual: 0.6454
Epoch [11/60], MSE: 0.0017, SSIM: 0.1334, Perceptual: 0.5775
Epoch [12/60], MSE: 0.0016, SSIM: 0.1294, Perceptual: 0.5767
Epoch [13/60], MSE: 0.0026, SSIM: 0.1627, Perceptual: 0.8075
Epoch [14/60], MSE: 0.0020, SSIM: 0.1454, Perceptual: 0.6078
Epoch [15/60], MSE: 0.0016, SSIM: 0.1421, Perceptual: 0.5903
Epoch [16/60], MSE: 0.0017, SSIM: 0.1156, Perceptual: 0.4647
Epoch [17/60], MSE: 0.0015, SSIM:

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# If a GPU is available, you can also check its details
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")


Using device: cuda
GPU Name: Tesla T4
GPU Memory Allocated: 0.00 GB
GPU Memory Cached: 0.00 GB
